In [1]:
from dlhpcstarter.utils import load_config_and_update_args
from dlhpcstarter.command_line_arguments import read_command_line_arguments
from longitudinal_model.modelling_longitudinal import LongitudinalPromptMultiCXREncoderDecoderModel, CvtWithProjectionHeadConfig
import torch
import transformers
import os
import warnings

/scratch2/nic261/environments/cxrmate/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transformers.__version__, torch.__version__

('4.36.2', '2.1.2+cu121')

In [3]:
# Hub checkpoint name:
hub_ckpt_name = 'aehrc/cxrmate-tf'

In [4]:
# Paths:

# Note: multi-image CXR report generation was named variable-image CXR report generation during development, hence, 091_any_prompt_variable_lora:
ckpt_path = '/datasets/work/hb-mlaifsp-mm/work/experiments/mimic_cxr/091_any_prompt_variable_lora/trial_0/epoch=6-step=27433-val_report_chexbert_f1_macro=0.388249.ckpt'
ckpt_zoo_dir = '/datasets/work/hb-mlaifsp-mm/work/checkpoints'

In [5]:
# Load state dict with depreciated keys:
state_dict = torch.load(ckpt_path, map_location=torch.device('cpu'))['state_dict']

In [6]:
# Encoder & decoder config:
config_decoder = transformers.BertConfig(
    vocab_size=30000,
    num_hidden_layers=6,
    type_vocab_size=2,
)  # BERT as it includes token_type_ids.
encoder_ckpt_name = 'microsoft/cvt-21-384-22k'
config_encoder = CvtWithProjectionHeadConfig.from_pretrained(
    os.path.join(ckpt_zoo_dir, encoder_ckpt_name),
    local_files_only=True,
    projection_size=config_decoder.hidden_size,
)
config = transformers.VisionEncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

# Encoder-to-decoder instance:
LongitudinalPromptMultiCXREncoderDecoderModel.register_for_auto_class("AutoModel")
encoder_decoder = LongitudinalPromptMultiCXREncoderDecoderModel(config=config)

trainable params: 147456 || all params: 80916528 || trainable%: 0.18223223814051934


/datasets/work/hb-mlaifsp-mm/work/repositories/cxrmate/modules/transformers/longitudinal_model/modelling_longitudinal.py:155: UserWarning: The encoder-to-decoder model was not warm-started before applying low-rank approximation.
  warnings.warn('The encoder-to-decoder model was not warm-started before applying low-rank approximation.')


In [7]:
for key in list(state_dict.keys()):
    if 'encoder_projection' in key:
        state_dict[key.replace('encoder_projection', 'encoder.projection_head.projection')] = state_dict.pop(key)
    elif 'last_hidden_state_layer_norm' in key:
        state_dict[key.replace('last_hidden_state_layer_norm', 'encoder.projection_head.layer_norm')] = state_dict.pop(key)
    elif 'encoder.encoder' in key:
        state_dict[key.replace('encoder.encoder', 'encoder.cvt.encoder')] = state_dict.pop(key)
    elif 'encoder_decoder.' in key:
        state_dict[key.replace('encoder_decoder.', '')] = state_dict.pop(key)
    else:
        warnings.warn(f'Key not found: {key}')

In [8]:
# Load renamed state dict:
encoder_decoder.load_state_dict(state_dict)

<All keys matched successfully>

In [9]:
# Save model:
save_path = '/scratch2/nic261/checkpoints/cxrmate/huggingface_longitudinal'
encoder_decoder.save_pretrained(save_path)

Removed shared tensor {'decoder.base_model.model.cls.predictions.decoder.weight', 'decoder.base_model.model.cls.predictions.decoder.bias'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


In [10]:
# Load tokenizer:
tokenizer_dir = os.path.join(ckpt_zoo_dir, 'mimic-cxr-tokenizers', 'bpe_prompt')
tokenizer = transformers.PreTrainedTokenizerFast.from_pretrained(tokenizer_dir, local_files_only=True)

In [11]:
# Image processor:
image_processor = transformers.AutoFeatureExtractor.from_pretrained(os.path.join(ckpt_zoo_dir, encoder_ckpt_name))
image_processor.save_pretrained(save_path)

/scratch2/nic261/environments/cxrmate/lib/python3.9/site-packages/transformers/models/convnext/feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


['/scratch2/nic261/checkpoints/cxrmate/huggingface_longitudinal/preprocessor_config.json']

In [12]:
# Hub login:
from huggingface_hub import login

with open('/home/nic261/hf_token.txt', 'r') as f:
    token = f.readline()
login(token=token[:-1])
del token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /scratch2/nic261/.cache/token
Login successful


In [13]:
# Push to hub:
encoder_decoder.push_to_hub(hub_ckpt_name)
tokenizer.push_to_hub(hub_ckpt_name)
image_processor.push_to_hub(hub_ckpt_name)

model.safetensors: 100%|██████████| 450M/450M [00:53<00:00, 8.36MB/s]    


CommitInfo(commit_url='https://huggingface.co/aehrc/cxrmate-tf/commit/47cab6a75721b3dee6369b33bda8a69a5ee946d1', commit_message='Upload feature extractor', commit_description='', oid='47cab6a75721b3dee6369b33bda8a69a5ee946d1', pr_url=None, pr_revision=None, pr_num=None)